In [1]:
import pandas as pd
import numpy as np

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
data = np.zeros((1000, 21))

protein_data = pd.read_csv('./EC number database_Archaea1.csv')
seq_data = protein_data.loc[:,['SEQ']].values # Extract the 'label' from 'dataFrame'.

In [3]:
seq_data = seq_data[0:7000]

In [4]:
non = []
t = 0
with open('123.txt', 'r') as f:
    for s in f:
        lenth = len(s)-1
        if t == 7000:
            break;
        non.append(s[1:lenth])      
        t = t + 1

In [5]:
aa_list = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X']

list_test = []
def split_data(data):
    list_test = []
    for i in range(data.shape[0]):
        list_test.append(list(data[i,0]))
        list_test.append([1])
    return list_test
    
def non_split_data(data):
    list_test = []
    for i in range(len(data)):
        list_test.append(list(data[i]))
        list_test.append([0])
    return list_test

tt = split_data(seq_data)
non_tt = non_split_data(non)

In [6]:
index_data = tt + non_tt
print(len(index_data))
print(index_data[27998])

28000
['M', 'R', 'T', 'L', 'W', 'M', 'A', 'L', 'C', 'A', 'L', 'S', 'R', 'L', 'W', 'P', 'G', 'A', 'Q', 'A', 'G', 'C', 'A', 'E', 'A', 'G', 'R', 'C', 'C', 'P', 'G', 'R', 'D', 'P', 'A', 'C', 'F', 'A', 'R', 'G', 'W', 'R', 'L', 'D', 'R', 'V', 'Y', 'G', 'T', 'C', 'F', 'C', 'D', 'Q', 'A', 'C', 'R', 'F', 'T', 'G', 'D', 'C', 'C', 'F', 'D', 'Y', 'D', 'R', 'A', 'C', 'P', 'A', 'R', 'P', 'C', 'F', 'V', 'G', 'E', 'W', 'S', 'P', 'W', 'S', 'G', 'C', 'A', 'D', 'Q', 'C', 'K', 'P', 'T', 'T', 'R', 'V', 'R', 'R', 'R', 'S', 'V', 'Q', 'Q', 'E', 'P', 'Q', 'N', 'G', 'G', 'A', 'P', 'C', 'P', 'P', 'L', 'E', 'E', 'R', 'A', 'G', 'C', 'L', 'E', 'Y', 'S', 'T', 'P', 'Q', 'G', 'Q', 'D', 'C', 'G', 'H', 'T', 'Y', 'V', 'P', 'A', 'F', 'I', 'T', 'T', 'S', 'A', 'F', 'N', 'K', 'E', 'R', 'T', 'R', 'Q', 'A', 'T', 'S', 'P', 'H', 'W', 'S', 'T', 'H', 'T', 'E', 'D', 'A', 'G', 'Y', 'C', 'M', 'E', 'F', 'K', 'T', 'E', 'S', 'L', 'T', 'P', 'H', 'C', 'A', 'L', 'E', 'N', 'W', 'P', 'L', 'T', 'R', 'W', 'M', 'Q', 'Y', 'L', 'R', 'E', 'G', 'Y'

In [7]:
def one_hot_encoding(split_data):
    seq_list = []
    for t in range(0, len(split_data), 2):
        data = np.zeros((1000, 21))
        test = split_data[t]
        for i in range(len(aa_list)):
            if len(test) > 1000:
                length_test = 1000
            else:
                length_test = len(test)
            for j in range(length_test):
                if aa_list[i] == test[j]:
                    data[j,i] = 1
        seq_list.append(data)
        test = []
        
    return seq_list

total_data = one_hot_encoding(index_data)

In [8]:
print(len(total_data))
print(total_data[2])

14000
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
class Share():
    def __init__(self, f_height = 1):
        super(Share, self).__init__()
        self.f_height = f_height
        conv1 = nn.Conv2d(1, 128, kernel_size = (self.f_height, 21), stride = 1) # 6@24*24
        batch_conv = nn.BatchNorm2d(128)
        pool1 = nn.MaxPool2d(kernel_size=(997,1)) # 6@12*a12
        
        self.conv_module = nn.Sequential(
            conv1,
            batch_conv,
            nn.ReLU(),
            pool1
        )

In [10]:
share = Share(4)
share2= Share()

In [11]:
share

In [12]:
class CNN1(nn.Module):
    def __init__(self):

        super(CNN1, self).__init__()
        
        share.conv_module # share객체 불러옴 conv, pool등 공유

        fc1 = nn.Linear(128, 512)
        fc2 = nn.Linear(512, 512)
        fc3 = nn.Linear(512, 2)
        # activation ReLU

        batch_fc1 = nn.BatchNorm1d(512)
        batch_fc2 = nn.BatchNorm1d(512)
        batch_fc3 = nn.BatchNorm1d(2)
        
        self.fc_module = nn.Sequential(
            fc1,
            batch_fc1,
            nn.ReLU(),
            fc2,
            batch_fc2,
            nn.ReLU(),
            fc3,
            batch_fc3,
            nn.Sigmoid()
        )
        
    def forward(self, x):
        out_list = []
        out = share.conv_module(x) # @16*4*4
        # make linear
        dim = 1
        for d in out.size()[1:]: #16, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return out

In [13]:
class CNN2(nn.Module):
    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작

        super(CNN2, self).__init__()
         
        # share객체 불러옴 conv, pool등 공유
        
        fc1 = nn.Linear(128, 512)
        fc2 = nn.Linear(512,512)
        fc2 = nn.Linear(512,9999)
        # activation ReLU

        self.fc_module = nn.Sequential(
            fc1,
            nn.ReLU(),
            fc2,
            nn.Sigmoid()
        )
        
    def forward(self, x):
        out_list = []
        out = self.conv_module(x) # @16*4*4
        dim = 1
        for d in out.size()[1:]: #16, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return out

In [14]:
class CNN3(nn.Module):
    def __init__(self):
        # 항상 torch.nn.Module을 상속받고 시작

        super(CNN3, self).__init__()

        share # share객체 불러옴 conv, pool등 공유
        
        fc1 = nn.Linear(128, 512)
        fc2 = nn.Linear(512,512)
        fc2 = nn.Linear(512,9999)
        # activation ReLU

        self.fc_module = nn.Sequential(
            fc1,
            nn.ReLU(),
            fc2,
            nn.Sigmoid()
        )
        
    def forward(self, x):
        out_list = []
        out = self.conv_module(x) # @16*4*4
        # make linear
        dim = 1
        for d in out.size()[1:]: #16, 4, 4
            dim = dim * d
        out = out.view(-1, dim)
        out = self.fc_module(out)
        return out

In [15]:
cnn1 = CNN1()
cnn2 = CNN2()
cnn3 = CNN3()

In [19]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.00001
optimizer1 = optim.Adam(cnn1.parameters(), lr=learning_rate)
optimizer2 = optim.Adam(cnn2.parameters(), lr=learning_rate)
optimizer3 = optim.Adam(cnn3.parameters(), lr=learning_rate)

In [ ]:
from torch.autograd import Variable
criterion = nn.CrossEntropyLoss()
# backpropagation method
# hyper-parameters
epochs = 9

for e in range(epochs):
    running_loss = 0
    t = 0
    for i in range(len(index_data)):
        if i % 2 == 1:
            
            
            cnn1.eval()
            index_value = torch.LongTensor(index_data[i])
            data = torch.from_numpy(total_data[t]).float()
            data = data.view(1,1,1000,21)
            optimizer1.zero_grad()
            model_output = cnn1(data)
            loss = criterion(model_output,index_value)
            loss.backward()
            # weight update
            optimizer1.step()
            t = t + 1 
            running_loss += loss.item()
        
    else:
        print("Epoch {} - Training loss: {}".format(e+1, running_loss/len(total_data)))


Epoch 1 - Training loss: 0.3475369003649269
Epoch 2 - Training loss: 0.487547026093517
Epoch 3 - Training loss: 0.49301486153474877
Epoch 4 - Training loss: 0.46366149501076764
Epoch 5 - Training loss: 0.490747682799186


In [ ]:
print(model_output)

print(index_value.shape)
#print(index_data[1])